In [ ]:
import gym
import pybullet_envs

env = gym.make('Walker2DBulletEnv-v0', render=True)
env.reset()

for _ in range(5000):
    action = env.action_space.sample()
    env.step(action)

env.close()


pybullet build time: Apr 14 2025 22:05:08


argv[0]=
Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-14 22:10:28.758 python[53015:8810502] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-14 22:10:28.758 python[53015:8810502] +[IMKInputSession subclass]: chose IMKInputSession_Modern


numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


: 

### Обучение агента

In [1]:
import gym
import pybullet_envs  # важно для регистрации сред
from stable_baselines3 import PPO

# Создание среды
env = gym.make("Walker2DBulletEnv-v0")

# Создание агента
model = PPO("MlpPolicy", env, verbose=1)

# Обучение агента (можно начать с 10000 шагов)
model.learn(total_timesteps=1000000)

# Сохранение модели
model.save("ppo_walker")
env.close()


pybullet build time: Apr 14 2025 22:05:08
/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
argv[0]=
argv[0]=
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.7     |
|    ep_rew_mean     | 13.5     |
| time/              |          |
|    fps             | 2044     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 13.2         |
|    ep_rew_mean          | 13.4         |
| time/                   |              |
|    fps                  | 1546         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0116548445 |
|    clip_fraction        | 0.116        |
|    clip_range           | 0.2 

KeyboardInterrupt: 

### Проверка обученной модели

In [1]:
import gym
import pybullet_envs
from stable_baselines3 import PPO
import time

# Загрузка обученной модели
model = PPO.load("ppo_walker")

# Создание среды с визуализацией
env = gym.make("Walker2DBulletEnv-v0", render=True)
obs = env.reset()

# Цикл визуализации
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(1.0 / 30.0)  # ~30 кадров в секунду — комфортная скорость

    if done:
        obs = env.reset()

env.close()


pybullet build time: Apr 14 2025 22:05:08


argv[0]=
Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-16 23:07:55.085 python[65796:9266809] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-16 23:07:55.085 python[65796:9266809] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

In [1]:
import pybullet as p
import pybullet_data
import time
import os

# GUI режим
physicsClient = p.connect(p.GUI)

# Загрузим стандартную плоскость и гравитацию
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.81)

# Путь к твоему URDF
robot_path = os.path.join("./berkeley_humanoid_description/urdf/", "robot.urdf")

# Загружаем твоего робота
robot_id = p.loadURDF(robot_path, [0, 0, 1])  # ставим выше земли

# Запускаем симуляцию
while True:
    p.stepSimulation()
    time.sleep(1.0 / 240.0)  # частота обновления


pybullet build time: Apr 14 2025 22:05:08


Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-15 00:28:18.963 python[55214:8878441] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-15 00:28:18.963 python[55214:8878441] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

### Создание кастомной среды под модель Berkeley Humanoid

In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import os
import time

class BerkeleyHumanoidEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 60}

    def __init__(self, render_mode=None):
        super().__init__()
        self.render_mode = render_mode
        self.time_step = 1. / 240.
        self.max_steps = 1000
        self.current_step = 0

        self.physics_client = p.connect(p.GUI if render_mode == "human" else p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)

        self.plane = p.loadURDF("plane.urdf")
        self.robot = p.loadURDF(os.path.join("./berkeley_humanoid_description/urdf/", "robot.urdf"), [0, 0, 1])

        self.joint_indices = [i for i in range(p.getNumJoints(self.robot)) if p.getJointInfo(self.robot, i)[2] != p.JOINT_FIXED]
        action_dim = len(self.joint_indices)
        self.action_space = spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(action_dim * 2,), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        p.resetSimulation()
        p.setGravity(0, 0, -9.81)
        p.loadURDF("plane.urdf")
        self.robot = p.loadURDF(os.path.join("./berkeley_humanoid_description/urdf/", "robot.urdf"), [0, 0, 1])
        self.joint_indices = [i for i in range(p.getNumJoints(self.robot)) if p.getJointInfo(self.robot, i)[2] != p.JOINT_FIXED]
        obs = self._get_obs()
        return obs, {}

    def _get_obs(self):
        obs = []
        for i in self.joint_indices:
            joint_state = p.getJointState(self.robot, i)
            obs.extend([joint_state[0], joint_state[1]])
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.current_step += 1

        for idx, joint_idx in enumerate(self.joint_indices):
            p.setJointMotorControl2(bodyIndex=self.robot,
                                    jointIndex=joint_idx,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=action[idx],
                                    force=50)

        p.stepSimulation()
        if self.render_mode == "human":
            time.sleep(self.time_step)

        obs = self._get_obs()
        base_pos, _ = p.getBasePositionAndOrientation(self.robot)
        reward = base_pos[0]  

        done = self.current_step >= self.max_steps
        truncated = False

        return obs, reward, done, truncated, {}

    def render(self):
        pass

    def close(self):
        p.disconnect()


pybullet build time: Apr 14 2025 22:05:08


In [8]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# Импорт среды
env = BerkeleyHumanoidEnv(render_mode="Human")
check_env(env)  # Проверка на совместимость с SB3

# Создание модели и обучение
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Сохранение
model.save("ppo_berkeley_humanoid")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -184     |
| time/              |          |
|    fps             | 1545     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | -93.3        |
| time/                   |              |
|    fps                  | 1261         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0144196255 |
|    clip_fraction        | 0.147        |
|    clip_range           | 0.2          |
|    en

### Проверка на модели робота 

In [2]:
import gym
import pybullet_envs
from stable_baselines3 import PPO
import time

model = PPO.load("ppo_berkeley_humanoid")
env = BerkeleyHumanoidEnv(render_mode="human")

obs, _ = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, truncated, _ = env.step(action)
    env.render()
    time.sleep(1.0 / 30.0)
    if done:
        obs, _ = env.reset()

env.close()

Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-16 23:08:32.001 python[65820:9267374] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-16 23:08:32.001 python[65820:9267374] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 